In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Operations").getOrCreate()
df = spark.read.csv("./house_apartment_cleaned_data.csv")
new_cols = ["locality", "type_of_property", "subtype_of_property", "price", "number_of_rooms", "open_fire", "terrace", "garden", "swimming_pool",
            "state_of_the_building", "construction_year"]
col_df = df.toDF(*new_cols)
# col_df.printSchema()
from pyspark.sql.functions import format_number
total_price = col_df.groupBy("subtype_of_property").agg({"price":"sum"})
min_val = total_price.orderBy("sum(price)", ascending=True).limit(1)
formatted_min_df = min_val.withColumn("sum(price)", format_number("sum(price)", 0))
max_val = total_price.orderBy("sum(price)", ascending=False).limit(1)
formatted_max_df = max_val.withColumn("sum(price)", format_number("sum(price)", 0))
final_df = formatted_max_df.union(formatted_min_df)
final_df.show()
final_df.rdd.getNumPartitions()
output_df = final_df.coalesce(1)
output_df.write.option("header", "true").parquet("./Que__1_parquet")

+-------------------+-------------+
|subtype_of_property|   sum(price)|
+-------------------+-------------+
|              house|6,125,912,375|
|           pavilion|       19,900|
+-------------------+-------------+

